<a href="https://colab.research.google.com/github/Mulongovictor/Machine-Learning-Projects/blob/main/22_MovieRecommendationSystemUsingSVD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 22 Movie Reccomendation System Using SVD 

# Importing the Basic Libraries

In [10]:
import numpy as np
import pandas as pd

# Importing and Parsing the Dataset as Ratings and Movies details

In [21]:
ratingData = pd.io.parsers.read_csv('ratings.dat', 
    names=['user_id', 'movie_id', 'rating', 'time'],
    engine='python', delimiter='::')


print(ratingData)

         user_id  movie_id  rating       time
0              1      1193       5  978300760
1              1       661       3  978302109
2              1       914       3  978301968
3              1      3408       4  978300275
4              1      2355       5  978824291
...          ...       ...     ...        ...
1000204     6040      1091       1  956716541
1000205     6040      1094       5  956704887
1000206     6040       562       5  956704746
1000207     6040      1096       4  956715648
1000208     6040      1097       4  956715569

[1000209 rows x 4 columns]


# Create the ratings matrix of shape (mxu)

In [22]:
ratingMatrix = np.ndarray(
    shape=(np.max(ratingData.movie_id.values), np.max(ratingData.user_id.values)),
    dtype=np.uint8)
ratingMatrix[ratingData.movie_id.values-1, ratingData.user_id.values-1] = ratingData.rating.values
print(ratingMatrix)

[[  5 141  50 ... 127   0   3]
 [128 161 220 ... 127   0   0]
 [ 64 116 221 ... 127   0   0]
 ...
 [  4   0   0 ...   0   0   0]
 [  5   0   0 ...   0   0   0]
 [  4   0   0 ...   0   0   0]]


# Subtract Mean off - Normalization

In [23]:
normalizedMatrix = ratingMatrix - np.asarray([(np.mean(ratingMatrix, 1))]).T
print(normalizedMatrix)

[[-61.27284768  74.72715232 -16.27284768 ...  60.72715232 -66.27284768
  -63.27284768]
 [ 31.73692053  64.73692053 123.73692053 ...  30.73692053 -96.26307947
  -96.26307947]
 [-20.11854305  31.88145695 136.88145695 ...  42.88145695 -84.11854305
  -84.11854305]
 ...
 [  3.52119205  -0.47880795  -0.47880795 ...  -0.47880795  -0.47880795
   -0.47880795]
 [  4.5013245   -0.4986755   -0.4986755  ...  -0.4986755   -0.4986755
   -0.4986755 ]
 [  3.32665563  -0.67334437  -0.67334437 ...  -0.67334437  -0.67334437
   -0.67334437]]


# Computing SVD

In [24]:
A = normalizedMatrix.T / np.sqrt(ratingMatrix.shape[0]-1)
U, S, V = np.linalg.svd(A)

# Calculate cosine similarity, sort by most similar and return the top N

In [25]:
def similar(ratingData, movie_id, top_n):
    index = movie_id - 1 # Movie id starts from 1
    movie_row = ratingData[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', ratingData, ratingData)) #Einstein summation |  traditional matrix multiplication and is equivalent to np.matmul(a,b)
    similarity = np.dot(movie_row, ratingData.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity) #Perform an indirect sort along the given axis (Last axis)
    return sort_indexes[:top_n]

# Select K Principal components to represent the movies, a movie_id to find recommendations and print the top_n results

In [28]:
movieData = pd.io.parsers.read_csv('movies.dat', names=['movie_id', 'title', 'genre'], engine='python', delimiter='::')

k = 50
movie_id = 2
top_n = 5

sliced = V.T[:, :k] # representative data
indexes = similar(sliced, movie_id, top_n)

print('Recommendations for Movie {0}: \n'.format(
movieData[movieData.movie_id == movie_id].title.values[0]))
for id in indexes + 1:
    print(movieData[movieData.movie_id == id].title.values[0])

UnicodeDecodeError: ignored